In [13]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump, load

In [14]:
input_x_train_file = "../datasets/drugs_x_train.csv"
input_y_train_file = "../datasets/drugs_y_train.csv"

df_x = pd.read_csv(input_x_train_file)
df_y = pd.read_csv(input_y_train_file)

df = pd.concat([df_x, df_y], axis=1)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2)
x_train

,Age,Na_to_K,Sex_F,Sex_M,BP_HIGH,BP_LOW,BP_NORMAL,Cholesterol_HIGH,Cholesterol_NORMAL
2,0.796610,0.322878,0,1,0,0,1,1,0
43,0.372881,0.179405,1,0,0,1,0,0,1
94,0.983051,0.377728,1,0,1,0,0,1,0
110,0.288136,0.037776,1,0,0,0,1,1,0
49,0.355932,0.327850,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...
89,0.322034,0.388830,0,1,1,0,0,1,0
50,0.762712,0.219964,1,0,1,0,0,1,0
40,0.542373,0.171931,1,0,0,1,0,1,0
30,0.322034,0.208081,1,0,0,1,0,0,1


In [16]:
x = x_train.values
y = y_train.values

clf = MLPClassifier(random_state=1, max_iter=1000)
clf.fit(x, y)

c:\Training\Language\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=1000, random_state=1)

In [17]:
preds = clf.predict(x_test.values)

acc_score = accuracy_score(preds, y_test.values)
acc_score

0.8888888888888888

In [18]:
model_file = "../models/clf_drugs.joblib"

dump(clf, model_file)

['../models/clf_drugs.joblib']